In [1]:
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
#  Input -> encoder -> gaussian (mean, std.dev) -> decoder -> reconstruction

class VAE(nn.Module):
    def __init__(self, in_shape,hidden_size, latent_size ):
        super(VAE, self).__init__()

        self.nin = np.prod(in_shape)
        self.latent_size = latent_size


        self.e_l1 = nn.Linear(self.nin, hidden_size)
        self.e_l2 = nn.Linear(hidden_size, hidden_size)
        self.mu = nn.Linear(hidden_size, self.latent_size)
        self.sigma  = nn.Linear(hidden_size, self.latent_size)

        self.d_l1 = nn.Linear(self.latent_size, self.hidden_size)
        self.d_l2 = nn.Linear(self.hidden_size, self.nin)
        

def forward(self, X):
    X = X.reshape(X.shape[0], -1)
    X = self.enc(self.e_l2(self.e_l1(X)))

    mu,sigma = self.mu(X), self.sigma(X)

    g = (mu + sigma) * torch.randn_like(sigma)

    out = self.d_l1(g)
    out = self.d_l2(g)

    return out




    



    return g 







    